In [1]:
# Crea un escenario de temperatura y precipitación
# futuro de acuerdo con el método Delta.

import os

import pandas as pd
import numpy as np

import xarray as xr

In [5]:
path_r = "../../../resultados/Delta/"
CHIRTS = "../../../resultados/CHIRTS/"
CHIRPS = "../../../datos/CHIRPS/"
Delta = "../../../resultados/AIRCC-CLIM/"
years = [2030, 2050, 2070]
RCP = ["RCP4p5", "RCP8p5"]
vars = ["Tmax", "Tmin", "Tmean", "Pre"]
var_delta = ["temp", "prec"]

# Si no existe la carpeta, la crea.
if not os.path.exists(path_r):
    os.mkdir(path_r)
for i in RCP:
    if not os.path.exists(
        path_r + i + "/"):
        os.mkdir(path_r + i + "/")
    for j in vars:
        if not os.path.exists(path_r
            + i + "/" + j + "/"):
            os.mkdir(path_r + i
                + "/" + j + "/")

In [6]:
fpath_pre = []
fpath_temp = []

# Crea una lista con todas las direcciones
# de los datos de temperatura.
for j in RCP:
    for y in years:
        for m in range(1, 13):
            fpath_temp.append(Delta + j + "/"
            + var_delta[0] + "_tol_Stochastic_"
            + j + "_" + f"{m:02d}" +
            "_" + str(y) + ".nc")

# Crea una lista con todas las direcciones
# de los datos de precipitación.
for j in RCP:
    for y in years:
        for m in range(1, 13):
            fpath_pre.append(Delta + j + "/"
            + var_delta[1] + "_tol_Stochastic_"
            + j + "_" + f"{m:02d}" +
            "_" + str(y) + ".nc")

In [7]:
# Calcula el Delta para la temperatura para cada año.

fpath = fpath_temp
vdelta = var_delta[0]

for v in vars[:-1]:
    # Abrimos un archivo como base para
    # concatenar, cuya variable sea compatible 
    # con los archivos actuales.
    path = CHIRTS + v
    ds_1 = xr.open_dataset(path + "/mexico_" 
    + v + ".2010.nc" ).isel(time = 0)
    ds_1 = ds_1.where(ds_1 > -9000)
    ds_1["time"] -= 1
    
    # Procesamos cada año.
    for j in range(2010, 2017):
        # Abrimos la información actual para
        # el año correspondiente.
        ds = xr.open_dataset(path + "/mexico_" 
            + v + "." + str(j) + ".nc" )
        ds = ds.where(ds > -9000)
        # Tenemos 6 conjuntos de deltas
        # mensuales, uno por cada año.
        for k in range (0, 6):
            # Copiamos el archivo base para concatenar.
            ds_2 = ds_1.copy()
            # Iteramos sobre un conjunto
            # de deltas mensuales.
            for i in range(k*12, (k+1)*12):
                # Abrimos un delta mensual.
                d = ( xr.open_dataset( fpath[i] )
                    .rename({vdelta: v}) )
                # Calculamos el delta para
                # el mes en cuestión.
                ds_dummy = ( ds.sel(time = ( ds["time.month"]
                    ==int(fpath_temp[i][-10:-8]) ) ) + d )
                # Concatenamos los deltas mensuales.
                ds_2 = xr.concat( [ds_2, ds_dummy],
                    dim = "time" )

            # Quitamos el prmer valor de nuestro archivo
            # de concatenación y guardamos el NetCDF.
            ds_2 = ds_2.drop_isel(time = 0)
            ds_2.to_netcdf( path_r + fpath[k*12][31:38]
                + v + "/mexico_" + v + "." +
                fpath_temp[k*12][-7:-2] + str(j) + ".nc" )

RuntimeError: NetCDF: HDF error

In [ ]:
# Calcula la temperatura para el periodo climatológico.

fpath = fpath_temp
vdelta = var_delta[0]

for v in vars[:-1]:
    # Abrimos un archivo como base para
    # concatenar, cuya variable sea compatible 
    # con los archivos actuales.
    path = CHIRTS
    ds_1 = xr.open_dataset(path + v + "/mexico_" 
    + v + ".2010.nc" ).isel(time = 0)
    ds_1 = ds_1.where(ds_1 > -9000)
    ds_1["time"] -= 1
    
    ds = xr.open_dataset(path + "/mexico_" 
        + v + "." + str(j) + ".nc" )
    ds = ds.where(ds > -9000)
    # Tenemos 6 conjuntos de deltas
    # mensuales, uno por cada año.
    for k in range (0, 6):
        # Copiamos el archivo base para concatenar.
        ds_2 = ds_1.copy()
        # Iteramos sobre un conjunto
        # de deltas mensuales.
        for i in range(k*12, (k+1)*12):
            # Abrimos un delta mensual.
            d = ( xr.open_dataset( fpath[i] )
                .rename({vdelta: v}) )
            # Calculamos el delta para
            # el mes en cuestión.
            ds_dummy = ( ds.sel(time = ( ds["time.month"]
                ==int(fpath_temp[i][-10:-8]) ) ) + d )
            # Concatenamos los deltas mensuales.
            ds_2 = xr.concat( [ds_2, ds_dummy],
                dim = "time" )

        # Quitamos el prmer valor de nuestro archivo
        # de concatenación y guardamos el NetCDF.
        ds_2 = ds_2.drop_isel(time = 0)
        ds_2.to_netcdf( path_r + fpath[k*12][31:38]
            + "mexico_" + v + "." +
            fpath_temp[k*12][-7:-2] + ".nc" )

In [ ]:
# Calcula el Delta para la precipitación para cada año.

v = vars[-1]
fpath = fpath_pre
vdelta = var_delta[1]
path = CHIRPS

# Abrimos un archivo como base para
# concatenar; para la precipitación
# se mantiene una sola variable.
ds_1 = xr.open_dataset(path + "/mexico_"
    + v + ".2010.nc" ).isel(time = 0)
ds_1 = ds_1.where(ds_1 > -9000)
ds_1["time"] -= 1
# Procesamos cada año.
for j in range(2010, 2017):
    # Abrimos la información actual para
    # el año correspondiente.
    ds = xr.open_dataset(path + "/mexico_"
        + v + "." + str(j) + ".nc" )
    ds = ds.where(ds > -9000)
    # Tenemos 6 conjuntos de deltas
    # mensuales, uno por cada año.
    for k in range (0, 6):
        # Copiamos el archivo base para concatenar.
        ds_2 = ds_1.copy()
        # Iteramos sobre un conjunto
        # de deltas mensuales.
        for i in range(k*12, (k+1)*12):
            # Abrimos un delta mensual.
            d = ( xr.open_dataset( fpath[i] )
                .rename({vdelta: v}) )
            # Calculamos el delta para
            # el mes en cuestión.
            ds_dummy = ( ds.sel(time = ( ds["time.month"]
                ==int(fpath_temp[i][-10:-8])
                ) ) * ( 1 + d ) )
            # Concatenamos los deltas mensuales.
            ds_2 = xr.concat( [ds_2, ds_dummy],
                dim = "time" )

        # Quitamos el prmer valor de nuestro archivo
        # de concatenación y guardamos el NetCDF.
        ds_2 = ds_2.drop_isel(time = 0)
        ds_2.to_netcdf( path_r + fpath[k*12][31:38]
            + v + "/mexico_" + v + "." +
            fpath_temp[k*12][-7:-2] + str(j) + ".nc" )